In [74]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')
 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
from src.lda_model import LDASmoothed 
from src.utils import get_vocab_from_docs, get_np_wct 
from pprint import pprint 

In [76]:
docs = [
    ['My name is tuo tuo .'],
    ['tuo tuo is cute !'],
    ['Tuo tuo likes to poo a lot !'],
    ['mama bear very tuo !']
]

docs = [doc[0].split(' ') for doc in docs]

print(docs)

[['My', 'name', 'is', 'tuo', 'tuo', '.'], ['tuo', 'tuo', 'is', 'cute', '!'], ['Tuo', 'tuo', 'likes', 'to', 'poo', 'a', 'lot', '!'], ['mama', 'bear', 'very', 'tuo', '!']]


In [77]:
wct_dict = get_vocab_from_docs(docs)
wct_dict 

{'My': [1, 0, 0, 0],
 'name': [1, 0, 0, 0],
 'is': [1, 1, 0, 0],
 'tuo': [2, 2, 1, 1],
 '.': [1, 0, 0, 0],
 'cute': [0, 1, 0, 0],
 '!': [0, 1, 1, 1],
 'Tuo': [0, 0, 1, 0],
 'likes': [0, 0, 1, 0],
 'to': [0, 0, 1, 0],
 'poo': [0, 0, 1, 0],
 'a': [0, 0, 1, 0],
 'lot': [0, 0, 1, 0],
 'mama': [0, 0, 0, 1],
 'bear': [0, 0, 0, 1],
 'very': [0, 0, 0, 1]}

In [78]:
wct_array, w_2_idx = get_np_wct(wct_dict, docs)

print(wct_array)
pprint(w_2_idx)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [2. 2. 1. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
{'!': 6,
 '.': 4,
 'My': 0,
 'Tuo': 7,
 'a': 11,
 'bear': 14,
 'cute': 5,
 'is': 2,
 'likes': 8,
 'lot': 12,
 'mama': 13,
 'name': 1,
 'poo': 10,
 'to': 9,
 'tuo': 3,
 'very': 15}


In [88]:
lda = LDASmoothed(
    docs,
    3, 
    wct_dict, 
    word_ct_array = wct_array
)
#LDASmoothed(docs, 3, wct_dict)

Topic Dirichlet Prior, Alpha
torch.Size([1, 3])
tensor([[1.0778, 1.1647, 1.0888]], dtype=torch.float64)

Word Dirichlet Prior, Eta
torch.Size([1, 16])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       dtype=torch.float64)

Var Inf - Word Dirichlet prior, Lambda
torch.Size([3, 16])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       dtype=torch.float64)

Var Inf - Topic Dirichlet prior, Gamma
torch.Size([4, 3])
tensor([[6.4111, 6.4980, 6.4221],
        [6.4111, 6.4980, 6.4221],
        [6.4111, 6.4980, 6.4221],
        [6.4111, 6.4980, 6.4221]], dtype=torch.float64)

Var -Inf - Word wise Topic Multinomial/Categorical, Phi
(4,)
[array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 